In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
from PIL import Image
import numpy as np
import os
from tqdm import tqdm

In [3]:
file = open("data/VGG-Face2/train_list.txt", "r")
train_txt = file.read().splitlines()
file.close()

In [4]:
labels = []
images = []
for i in range(len(train_txt)):
    entry = train_txt[i].split('/')
    labels.append(entry[0])
    #images.append(entry[1])

len(labels)

3141890

In [5]:
from collections import Counter
label_counts = Counter(labels)
labels = np.array(list(label_counts.keys()))
counts = np.array(list(label_counts.values()))
large_classes = labels[counts > 500]
len(large_classes)

792

In [6]:
train_data_root = "data/VGG-Face2/vggface2_train/train/"

In [7]:
df = pd.read_pickle("MAAD_Face_1.0.pkl")
#df = pd.read_csv("MAAD_Face.csv")

In [8]:
#data preprocessing
df.at[250634, 'Identity'] = 713
df[["Identity", "Male"]] = df[["Identity", "Male"]].apply(pd.to_numeric)

folder_list = []
for i in range(len(df)):
    folder_list.append(df['Filename'][i].split('/')[0])

df['Folder'] = folder_list

In [9]:
df

,Filename,Identity,Male,Young,Middle_Aged,Senior,Asian,White,Black,Rosy_Cheeks,...,Pointy_Nose,Heavy_Makeup,Wearing_Hat,Wearing_Earrings,Wearing_Necktie,Wearing_Lipstick,No_Eyewear,Eyeglasses,Attractive,Folder
0,n000002/0001_01.jpg,2,-1,1,-1,-1,-1,1,-1,0,...,1,1,-1,1,-1,1,1,-1,1,n000002
1,n000002/0002_01.jpg,2,-1,1,-1,-1,-1,1,-1,0,...,1,1,-1,1,-1,1,1,-1,1,n000002
2,n000002/0003_01.jpg,2,-1,1,-1,-1,-1,1,-1,0,...,1,1,-1,1,-1,1,1,-1,1,n000002
3,n000002/0004_01.jpg,2,-1,1,-1,-1,-1,1,-1,0,...,1,1,-1,1,-1,1,1,-1,1,n000002
4,n000002/0005_01.jpg,2,-1,1,-1,-1,-1,1,-1,0,...,1,1,-1,1,-1,1,1,-1,1,n000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3308035,n009294/0284_01.jpg,9294,-1,0,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,1,1,-1,-1,n009294
3308036,n009294/0287_01.jpg,9294,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,1,1,-1,-1,n009294
3308037,n009294/0289_01.jpg,9294,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,0,-1,1,0,-1,-1,n009294
3308038,n009294/0290_02.jpg,9294,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,0,-1,1,1,-1,-1,n009294


In [10]:
large_classes_ids = []
for i in range(len(large_classes)):
    large_classes_ids.append(int(large_classes[i][3:]))

In [11]:
large_classes_ids = np.array(large_classes_ids)

In [12]:
def select_balanced_classes(Nclasses, large_classes_ids, attribute='Male'):
    #theoretically should throw error if replace=False and the sample size is greater than the population size
    labels = []

    for i in tqdm(range(len(large_classes_ids))):
        label = df[df['Identity'] == large_classes_ids[i]][attribute].mode() #gets the most common label for this identity
        label = label.to_list()[0]
        labels.append(label)
    
    labels=np.array(labels)

    class_1 = large_classes_ids[labels == -1]
    class_2 = large_classes_ids[labels == 1]

    class_1_subsampled = np.random.choice(class_1, Nclasses, replace=False)
    class_2_subsampled = np.random.choice(class_2, Nclasses, replace=False) #NEED TO RERUN THIS CODE NOW THAT I REPLACE THINGS AHHHHHHH

    #filtered_df = df[df['Identity'].isin(large_classes_ids)]
    #class_1_df = filtered_df[filtered_df[attribute] == -1]
    #class_2_df = filtered_df[filtered_df[attribute] == 1]
    #class_1_subsampled = class_1_df['Identity'].sample(n=Nclasses).to_list()
    #class_2_subsampled = class_2_df['Identity'].sample(n=Nclasses).to_list()
    
    train_identities = np.concatenate((class_1_subsampled[0:int(Nclasses/2)], class_2_subsampled[0:int(Nclasses/2)]))
    test_identities = np.concatenate((class_1_subsampled[int(Nclasses/2):], class_2_subsampled[int(Nclasses/2):]))
    train_labels = [0] * int(Nclasses/2) + [1] * int(Nclasses/2) #output 0s and 1s instead of -1 and 1
    return(train_identities, test_identities, train_labels)


        

        

In [13]:
#lacuna10_train_ids, lacuna10_ood_ids, lacuna10_train_labels = select_balanced_classes(10, large_classes_ids, attribute='Male')
lacuna100_train_ids, lacuna100_ood_ids, lacuna100_train_labels = select_balanced_classes(100, large_classes_ids, attribute='Male')

100%|██████████| 792/792 [00:01<00:00, 445.57it/s]


In [16]:
#selected_classes = np.random.choice(large_classes, 110)

#lacuna100 = selected_classes[:100]
#lacuna10 = selected_classes[:10]



def get_image(image_path, resize_to=None):
    image = Image.open(image_path)
    if resize_to is not None:
        image = image.resize(resize_to)

    #image = np.expand_dims(image, axis=0)
    return np.array(image)

def make_train_test_dataset(data_root, classes, labels, resize_to=None, num_samples=500, dest="data/lacuna100binary128/in_distribution"): #classes is list of identities, labels is binary label

    try:
        os.makedirs(os.path.join(dest,'train'))
        os.makedirs(os.path.join(dest,'test'))
    except:
        pass

    dataset_train = None
    targets_train = None
    identities_train = None
    dataset_test = None
    targets_test = None
    identities_test = None

    N_train = 0
    N_test = 0

    for idx in tqdm(range(len(classes))):
        identity = classes[idx]
        
        folder = df[df['Identity'] == identity]['Folder'].iloc[0]

        images = []
        for fil in os.listdir(os.path.join(train_data_root, folder)):
            if fil.endswith('.jpg'):
                images.append(fil) #filenames
        selected_images = np.random.choice(images, num_samples)

        #train test split
        selected_images_train = selected_images[:400]
        selected_images_test = selected_images[400:]


        identity = np.array([identity])
        identity = np.expand_dims(identity, axis=0)

        for j in range(len(selected_images_train)):
            img = selected_images_train[j]
            image_path = os.path.join(data_root, folder, img)
            image = get_image(image_path, resize_to)

            N = idx*len(selected_images_train) + j
            np.save(os.path.join(dest, "train", f'image{N}.npy'), image)
            
            N_train += 1
            

            label = np.array([labels[idx]])
            label = np.expand_dims(label, axis=0)


            if targets_train is None:
                targets_train = label
            else:
                targets_train = np.concatenate((targets_train,label), axis=0)

            if identities_train is None:
                identities_train = identity
            else:
                identities_train = np.concatenate((identities_train,identity), axis=0)


        for j in range(len(selected_images_test)):
            img = selected_images_test[j]
            image_path = os.path.join(data_root, folder, img)
            image = get_image(image_path, resize_to)

            N = idx*len(selected_images_test) + j
            np.save(os.path.join(dest, "test", f'image{N}.npy'), image)
            N_test += 1


            if targets_test is None:
                targets_test = label
            else:
                targets_test = np.concatenate((targets_test,label), axis=0)

            if identities_test is None:
                identities_test = identity
            else:
                identities_test = np.concatenate((identities_test,identity), axis=0)


    if targets_train is not None and identities_train is not None:
        
        np.save(os.path.join(dest, "train", 'label.npy'), targets_train.reshape(-1))
        np.save(os.path.join(dest, "train", 'identities.npy'), identities_train.reshape(-1))
        print ("OK! train set saved")
        print ("dataset size: {}\tlabels size: {}\tidentities size: {}".format(N_train, targets_train.shape, identities_train.shape))
    else:
        print ("Error! train set did not saved as the sizes are zero")

    if targets_test is not None and identities_test is not None:
        np.save(os.path.join(dest, "test", 'label.npy'), targets_test.reshape(-1))
        np.save(os.path.join(dest, "test", 'identities.npy'), identities_test.reshape(-1))
        print ("OK! test set saved")
        print ("dataset size: {}\tlabels size: {}\tidentities size: {}".format(N_test, targets_test.shape, identities_test.shape))
    else:
        print ("Error! test set did not saved as the sizes are zero")
    
    np.save(os.path.join(dest, 'identities.npy'), classes)
    print(image.shape)
    print("Generated dataset with IDs :" + str(classes.tolist()))

In [17]:
def make_dataset(data_root, classes, labels, resize_to=None, num_samples=100, dest="data/lacuna100binary128/oo_distribution"): #classes is list of identities, labels is binary label

    try:
        os.makedirs(dest)
    except:
        pass

    dataset = None
    targets = None
    identities = None

    N_samples = 0

    for idx in tqdm(range(len(classes))):
        identity = classes[idx]
        
        folder = df[df['Identity'] == identity]['Folder'].iloc[0]

        images = []
        for fil in os.listdir(os.path.join(train_data_root, folder)):
            if fil.endswith('.jpg'):
                images.append(fil) #filenames
        selected_images = np.random.choice(images, num_samples)

        identity = np.array([identity])
        identity = np.expand_dims(identity, axis=0)

        for j in range(len(selected_images)):
            img = selected_images[j]
            image_path = os.path.join(data_root, folder, img)
            image = get_image(image_path, resize_to)

            N = idx*len(selected_images) + j
            np.save(os.path.join(dest, f'image{N}.npy'), image)
            
            N_samples += 1

            label = np.array([labels[idx]])
            label = np.expand_dims(label, axis=0)


            if targets is None:
                targets = label
            else:
                targets = np.concatenate((targets,label), axis=0)

            if identities is None:
                identities = identity
            else:
                identities = np.concatenate((identities,identity), axis=0)

    if targets is not None and identities is not None:
        
        np.save(os.path.join(dest, 'label.npy'), targets.reshape(-1))
        np.save(os.path.join(dest, 'identities.npy'), identities.reshape(-1))
        print ("OK! dataset saved")
        print ("dataset size: {}\tlabels size: {}\tidentities size: {}".format(N_samples, targets.shape, identities.shape))
    else:
        print ("Error! dataset did not save as the sizes are zero")
    
    np.save(os.path.join(dest, 'total_identities.npy'), classes)
    print(image.shape)
    print("Generated dataset with IDs :" + str(classes.tolist()))


In [20]:
lacuna100_train_ids

array([3344, 1091, 2066, 7590, 1397, 7564,  326, 7785, 4038, 5898, 2397,
       1237, 4844, 2297, 8930, 8545,  563,  628, 3478, 3181, 3210, 7981,
       7980, 8316, 8186, 2360, 2366, 2115, 1727, 8879, 2618, 2502,  472,
       2447, 3615, 4373, 7129, 7120,  105, 3973, 7688, 5779, 8502, 7610,
       5249, 4150, 5014, 5901, 3392, 3107, 5335, 3680, 2350, 7327, 3831,
       4178, 4940, 1132, 8901, 7392, 1496, 7938, 5292, 3776, 4598, 7540,
       3769, 5483, 7106, 8683, 1642, 4001, 2060,  560, 1369, 3651, 4686,
       7353, 6352,  956, 9036, 1052, 3170, 7658, 5343, 3520, 6671, 5828,
       3485,  598, 5814, 3735, 2142, 5738, 5491, 1200, 2141, 6011, 8658,
       3239])

In [25]:
len(set(lacuna100_ood_ids))

100

In [22]:
make_train_test_dataset(train_data_root, lacuna100_train_ids, lacuna100_train_labels, resize_to=(128, 128))


100%|██████████| 100/100 [01:17<00:00,  1.29it/s]

OK! train set saved
dataset size: 40000	labels size: (40000, 1)	identities size: (40000, 1)
OK! test set saved
dataset size: 10000	labels size: (10000, 1)	identities size: (10000, 1)
(128, 128, 3)
Generated dataset with IDs :[3344, 1091, 2066, 7590, 1397, 7564, 326, 7785, 4038, 5898, 2397, 1237, 4844, 2297, 8930, 8545, 563, 628, 3478, 3181, 3210, 7981, 7980, 8316, 8186, 2360, 2366, 2115, 1727, 8879, 2618, 2502, 472, 2447, 3615, 4373, 7129, 7120, 105, 3973, 7688, 5779, 8502, 7610, 5249, 4150, 5014, 5901, 3392, 3107, 5335, 3680, 2350, 7327, 3831, 4178, 4940, 1132, 8901, 7392, 1496, 7938, 5292, 3776, 4598, 7540, 3769, 5483, 7106, 8683, 1642, 4001, 2060, 560, 1369, 3651, 4686, 7353, 6352, 956, 9036, 1052, 3170, 7658, 5343, 3520, 6671, 5828, 3485, 598, 5814, 3735, 2142, 5738, 5491, 1200, 2141, 6011, 8658, 3239]


In [23]:
make_dataset(train_data_root, lacuna100_ood_ids, lacuna100_train_labels, resize_to=(128, 128))


100%|██████████| 100/100 [00:16<00:00,  6.03it/s]

OK! dataset saved
dataset size: 10000	labels size: (10000, 1)	identities size: (10000, 1)
(128, 128, 3)
Generated dataset with IDs :[5408, 1329, 3121, 6391, 419, 3374, 1178, 7559, 1221, 1659, 2724, 248, 3701, 7280, 9161, 4617, 7001, 1333, 4949, 3716, 2454, 2507, 7971, 8638, 5176, 6239, 5338, 2648, 4151, 4651, 6573, 3746, 6623, 6400, 771, 8023, 5782, 2909, 3815, 6860, 5496, 6379, 5318, 111, 228, 4439, 8048, 2947, 6394, 1322, 6138, 4257, 6481, 8972, 1592, 7445, 257, 5674, 7174, 2269, 4270, 6363, 4275, 9052, 4984, 176, 241, 7009, 2262, 554, 103, 2789, 3938, 2885, 7616, 5384, 3076, 4124, 857, 2278, 5829, 8242, 3965, 3350, 817, 3047, 7139, 1367, 5677, 7493, 1493, 8633, 8524, 1775, 8529, 547, 185, 9194, 8078, 4550]
